# Árboles de Regresión

En este notebook vamos a retomar los datos de Facebook con los que trabajamos en la tercera sección y vamos a construir sobre ellos árboles de regresión para predecir de nuevo el número de likes de una publicación dadas algunas características como el tipo de publicación o la hora de publicación. En este modelo haremos especial hincapié en la explicabilidad interpretando los árboles generados. 

Para el desarrollo de este notebook emplearemos los siguientes módulos:

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

## Carga de datos

Como siempre comenzamos cargando los datos. Cargamos los datos de Facebook ya limpios y procesados. Si no recuerdas cómo se realizó este preprocesamiento puedes revisar el notebook de la regresión lineal del tema 3.

In [ ]:
data_fb = pd.read_csv('./data/cleaned_facebook.csv')
data_fb.head()

In [ ]:
X = data_fb.drop('like', axis=1)
y = data_fb['like']

Dividimos como siempre nuestros datos en datos de entrenamiento y datos de validación:

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21)

## Construcción del modelo

Una vez dispuestos los datos comenzamos con la construcción del modelo. Usaremos de nuevo el módulo scikit-learn. Comenzamos instanciando el modelo:

In [ ]:
from sklearn import tree
tree_one = tree.DecisionTreeRegressor()

Una vez instanciado el modelo con las especificaciones que vienen por defecto ajustamos el modelo empleando los datos de entrenamiento:

In [ ]:
tree_one = tree_one.fit(X_train, y_train)

Con estas dos líneas de código hemos construido nuestro primer árbol de regresión. Podemos observar su profundidad:

In [ ]:
tree_one.get_depth()

O el número mínimo de observaciones para producir una separación:

In [ ]:
tree_one.min_samples_split

Recordemos que una de las partes más importantes de los árboles es que podemos visualizar la estructura:

In [ ]:
tree.plot_tree(tree_one,
               feature_names = X_train.columns, 
               filled = True);

Observamos que el árbol generado es terriblemente complejo. Es muy probable que se esté produciendo overfitting, vamos a comprobarlo empleando el estadístico R2:

In [ ]:
y_pred_train = tree_one.predict(X_train)

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_train, y_pred_train)

In [ ]:
y_pred_test = tree_one.predict(X_test)
r2_score(y_test, y_pred_test)

Efectivamente nos encontramos ante un caso de overfitting enorme. El modelo es casi perfecto en entrenamiento (recordemos que la perfección en R2 se estima como 1) y muy muy malo en validación con un R2 negativo. 

Esto no sorprende, sabemos que los árboles tienden a sobreajustarse cuando se complican demasiado. Generamos a continuación un árbol en el que controlaremos la complejidad.

Instanciamos un nuevo modelo en el que fijamos una profundidad máxima de 2 y un número mínimo de muestras para separación de 4:

In [ ]:
tuned_tree = tree.DecisionTreeRegressor(max_depth=2, min_samples_split=4)

Ajustamos este nuevo modelo a los datos de entrenamiento:

In [ ]:
tuned_tree = tuned_tree.fit(X_train, y_train)

De nuevo evaluamos su calidad y comprobamos el overfitting mediante el estadístico R2:

In [ ]:
y_pred_tuned_train = tuned_tree.predict(X_train)
r2_score(y_train, y_pred_tuned_train)

In [ ]:
y_pred_tuned_test = tuned_tree.predict(X_test)
r2_score(y_test, y_pred_tuned_test)

Este modelo es aun mejorable. Te animo a probar nuevas configuraciones que puedan mejorarlo. Una vez elegido nuestro modelo podemos calcular el error medio cuadrático para poder compararlo con otros modelos como el de regresión lineal:

In [ ]:
from sklearn.metrics import mean_squared_error
mse_train = mean_squared_error(y_train, y_pred_tuned_train)
np.sqrt(mse_train)

In [ ]:
mse_test = mean_squared_error(y_test, y_pred_tuned_test)
np.sqrt(mse_test)

Obtenemos unos resultados catastróficos. Esto tiene sentido, recordemos que la regresión lineal nos devolvía ya unos resultados bastante malos y en los árboles estamos sacrificando precisión a cambio de explicabilidad por lo que estos resultados no sorprenden tanto. Pero veamos la gran ventaja. Ahora podemos representar gráficamente los árboles y clasificar cualquier nuevo elemento mediante solo 2 comprobaciones:

In [ ]:
import graphviz
# DOT data
dot_data = tree.export_graphviz(tuned_tree, out_file=None, 
                                feature_names=X_train.columns,  
                                filled=True)

# Draw graph
graph = graphviz.Source(dot_data, format="png") 
graph

Así para cualquier nueva observación bastara comprobar si es de categoría 1 y en caso de serlo si fue publicada en septiembre o no. En caso de no serlo si fue publicada de miércoles o no. Con solo estas dos preguntas podemos generar nuestra predicción. Esto además nos permite tomar decisiones, por ejemplo si tenemos un post de categoría uno y estamos pensando en publicarlo en septiembre quizá sea mejor esperar a octubre donde la predicción es de 170 likes más. Este es el mayor interés de los árboles, no tanto su precisión si no la conclusiones que se pueden sacar a partir de sus bifurcaciones así como la sencillez para nuevas inferencias que puede ser de una enorme utilidad si pensamos por ejemplo en decisiones en salas de espera de hospitales.